In [1]:
import pandas as pd
import numpy as np
import json
from haversine import haversine
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))
# Email from Bruce, 2019-05-30 4:16 p.m.
# Life's Occasions test primary zips

store_list=['49','1243','1817','5340','5354']

In [2]:
clients_zip=pd.ExcelFile("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/zips_Cincinnati_Bruce_20190530/Cincinnati Life's Occasions Test Trade Areas.xlsx")

clients_zip=clients_zip.parse("Sheet1",dtype=str,skiprows=4,usecols=[1,2])
clients_zip=clients_zip.head(70)

df_client_stores=pd.DataFrame()
for store_id,group in clients_zip.groupby("PrimStore"):
    if store_id!="nan":
        df_client_stores=df_client_stores.append(group)
df_client_stores=df_client_stores.rename(columns={"PrimStore":"location_id"}) 
df_client_stores['client_inclusion']=1

In [3]:
df_client_stores.head(2)

,postal_cd,location_id,client_inclusion
8,47025,1243,1
9,47001,1243,1


In [4]:
store_zips_labels=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv",
                              dtype=str,usecols=['location_id','zip','revenue_flag'])
store_zips_labels=store_zips_labels.rename(columns={"zip":'postal_cd','revenue_flag':'juba_flag'})
store_zips_labels=store_zips_labels[store_zips_labels['location_id'].isin(df_client_stores['location_id'].tolist())]
store_zips_labels=store_zips_labels[store_zips_labels['juba_flag'].isin(["P","S"])]

In [5]:
df_compare=pd.merge(df_client_stores,store_zips_labels,on=["postal_cd",'location_id'],how="outer")
df_compare_1=df_compare[df_compare['location_id'].isin(store_zips_labels['location_id'].unique().tolist())]
df_compare_2=df_compare[~df_compare['location_id'].isin(store_zips_labels['location_id'].unique().tolist())]


In [6]:
df_compare_1.shape

(53, 4)

In [7]:
store_locations=pd.read_csv("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190501-135143-036.txt",
                            sep="|",dtype=str,usecols=['location_id','latitude_meas','longitude_meas'])
store_locations=store_locations[store_locations['location_id'].isin(df_compare_2['location_id'].tolist())]
store_locations['latitude_meas']=store_locations['latitude_meas'].astype(float)
store_locations['longitude_meas']=store_locations['longitude_meas'].astype(float)
store_locations=store_locations.reset_index()
del store_locations['index']

df_2_new_stores=pd.DataFrame()
for index,row in store_locations.iterrows():
    store_center=(row['latitude_meas'],row['longitude_meas'])
    for zip_cd in zip_centers.keys():
        dist=haversine(zip_centers[zip_cd],store_center,miles=True)
        if dist<=10:
            df=pd.DataFrame({"location_id":row['location_id'],"postal_cd":zip_cd,"juba_flag":'zips_in_10'},index=[0])
            df_2_new_stores=df_2_new_stores.append(df)
store_zips_labels=store_zips_labels.append(df_2_new_stores)

df_compare=pd.merge(df_client_stores,store_zips_labels,on=["postal_cd",'location_id'],how="outer")


In [10]:
import datetime
writer=pd.ExcelWriter("./BL_comepare_zip_5_Cincinnati_stores_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_compare.to_excel(writer,"df_compare",index=False)
writer.save()